In [11]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt

In [12]:
sys.path.append('../readmaroc')

In [13]:
%matplotlib qt

In [14]:
from maroc_data import MarocData

In [15]:
from scipy.stats import norm

In [16]:
from functools import partial

In [17]:
import matplotlib.backends.backend_pdf

In [18]:
input_dat = '../data/0806/Run000092044.dat'

In [19]:
marocdata = MarocData(input_dat)

In [20]:
y_offset = [12000, 10000, 8000, 4000, 2000]

In [21]:
def overthreshold(marocdata, event_id):
    events = marocdata.get_event(event_id)
    count = 0
    for board_id, event in events:
        
        if event is None:
            continue
        board = marocdata.get_board(board_id)
        pedestal = board.avg_data
        noise = board.noise
        mu, std = norm.fit(noise)
        signal = event.signal
        if np.any(signal-pedestal>=200+(mu+5*std)):
            count+=1
    if count>=2:
        return True
    return False

In [22]:
event_ids_over_theshold = list(filter(partial(overthreshold, marocdata), range(1, marocdata.max_evt+1)))

In [23]:
len(event_ids_over_theshold)

23

In [24]:
def plot_event(evt, marocdata):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16,10), sharey=True, sharex=True)
    for i, (triplet_y, triplet_x) in enumerate(zip(np.arange(1, 16).reshape(5, 3), 
                                                   np.arange(16,31).reshape(5,3))):
        for j, (board_y, board_x) in enumerate(zip(triplet_y, triplet_x)):
            if board_y in marocdata.active_boards:
                if evt in marocdata.get_board(board_y):
                    signal = marocdata.get_board(board_y).get_event(evt).signal
                    ax1.text((320+320*j-0+(j*320))/2 -150, y_offset[i]-250, 'board {}, TS: {} '.format(board_y, marocdata.get_board(board_y).clean_timestamps[evt]), size='small')
                    if np.max(signal)>2000:
                        signal = signal*0.45
                    ax1.plot(np.arange(0+(j*320), 320+320*j), 
                             (signal)+y_offset[i], color='blue', linewidth=1)
                    ax1.axvline(320*(j+1), linestyle='--', linewidth=.75, color='grey', alpha=.5)
                else:
                    pass
            if board_x in marocdata.active_boards:
                if evt in marocdata.get_board(board_x):
                    signal = marocdata.get_board(board_x).get_event(evt).signal
                    ax2.text((320+320*j-0+(j*320))/2 -150, y_offset[i]-250, 'board {}, TS:{} '.format(board_x, marocdata.get_board(board_x).clean_timestamps[evt]), size='small')
                    if np.max(signal)>2000:
                        signal = signal*0.45
                    ax2.plot(np.arange(0+(j*320), 320+320*j), 
                             (signal)+y_offset[i], color='red', linewidth=1)
                    ax2.axvline(320*(j+1), linestyle='--', linewidth=.75, color='grey', alpha=.5)
                else:
                    pass
            
    ax1.set_title('y layers', size='x-large')
    ax2.set_title('x layers', size='x-large')
    plt.yticks(y_offset, ['layer 0','layer 1', 'layer 2', 'layer 3', 'layer 4'], size='x-large')
    plt.xticks([0, 320, 640, 960])
    fig.text(.5, .05, 'strips', size='large')
    fig.text(.5, .95, 'Evt {}'.format(evt), size='large')
    plt.ylim(1000, 13000)
    return fig, ax1, ax2



In [25]:
for evt in event_ids_over_theshold:
    fig, ax1, ax2 = plot_event(evt, marocdata)
    plt.show()
    break

In [26]:
pdf = matplotlib.backends.backend_pdf.PdfPages("output_2hits_TS_clean.pdf")
for evt in event_ids_over_theshold:
    fig, ax1, ax2 = plot_event(evt, marocdata)
    pdf.savefig(fig)
    plt.close(fig)
pdf.close()
plt.close()